### Geo Foundation Model
https://arxiv.org/pdf/2302.04476

For pretraining, we employ 8 NVIDIA V100 GPUs with
a batch size of 2048 (128 per GPU) and the image size
of 192×192. All pretraining settings are the same as in
[43]. For downstream tasks, 4 NVIDIA A10G GPUs are
employed. During the pretraining stage, we utilize RGB
bands as they are most commonly available among data
sources and tasks.

### nb by looking for maximum in submission file i found that there are 1540x1540 tiles

In [1]:
import os

os.listdir('.')

['eyeball_validate_train_utils.ipynb',
 'etl.py',
 'playground.ipynb',
 'eda.ipynb',
 'wandb',
 'train.ipynb',
 'etl.ipynb',
 'inference.ipynb']

In [2]:
import torch

# Path to your gfm.pth model
model_path = 'simmim_pretrain/gfm.pth'  # Replace with the actual path

try:
    # Attempt to load the entire model
    # Use 'cuda' if GPU is available
    model = torch.load(model_path, map_location='cpu')
    print("Model loaded successfully!")
    print(model)
except Exception as e:
    print(f"Failed to load model: {e}")

Failed to load model: [Errno 2] No such file or directory: 'simmim_pretrain/gfm.pth'


/tmp/ipykernel_279227/3433952129.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location='cpu')


In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import timm  # PyTorch Image Models library

# Step 1: Instantiate the Swin Transformer model
model = timm.create_model(
    'swin_base_patch4_window7_224',  # Base model; adjust if necessary
    pretrained=False,
    num_classes=0  # Set to 0 for feature extraction; adjust as needed
)

# Step 2: Load the checkpoint
model_path = '../simmim_pretrain/gfm.pth'  # Replace with the actual path
checkpoint = torch.load(model_path, map_location='cpu')
print("Checkpoint keys:", checkpoint.keys())

# Step 3: Extract and clean the state dictionary
state_dict = checkpoint['model']
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('module.'):
        new_state_dict[k[len('module.'):]] = v
    else:
        new_state_dict[k] = v

# Step 4: Load the state dictionary into the model
# Use strict=True if you are sure of the match
model.load_state_dict(new_state_dict, strict=False)
model.eval()

# Step 5: Prepare a random image tensor
batch_size = 1
channels = 3
height = 224  # Match pretraining image size
width = 224   # Match pretraining image size

random_image = torch.randn(batch_size, channels, height, width)

# Optional: Apply normalization
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
random_image = normalize(random_image)

# Step 6: Perform a forward pass
with torch.no_grad():
    output = model(random_image)

print("input shape:", random_image.shape)
print("Model Output:", output)
print("Model Output Shape:", output.shape)

/tmp/ipykernel_279227/1916446933.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')


Checkpoint keys: dict_keys(['model', 'optimizer', 'lr_scheduler', 'max_accuracy', 'epoch', 'config', 'amp'])
input shape: torch.Size([1, 3, 224, 224])
Model Output: tensor([[-0.4588,  0.1096, -0.6445,  ..., -0.0775,  0.5360,  1.1420]])
Model Output Shape: torch.Size([1, 1024])


In [4]:
# ============================================
# GeoAI Amazon Basin Secret Runway Detection
# Data Inspection Script
# ============================================

# ----------------------------
# 1. Environment Setup
# ----------------------------

# Install required libraries (Uncomment if running for the first time)
# !pip install geopandas matplotlib pandas

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys

# ----------------------------
# 2. Inspecting Training Data
# ----------------------------

# a. Define the Path to the Training Shapefile
training_shapefile_path = os.path.join(
    'pac_2024_training', 'pac_2024_training.shp')

if not os.path.exists(training_shapefile_path):
    print(f"Training shapefile not found at: {training_shapefile_path}")
    sys.exit(1)

# b. Load the Training Shapefile
training_data = gpd.read_file(training_shapefile_path)
print("Training Data Loaded Successfully!")

# c. Inspect the First Few Rows
print("\nFirst Five Rows of Training Data:")
print(training_data.head())

# d. Check Data Structure and CRS
print("\nTraining Data Information:")
print(training_data.info())

print("\nTraining Data CRS:")
print(training_data.crs)

# e. Explore Attribute Columns
print("\nTraining Data Columns:")
print(training_data.columns)

if 'year' in training_data.columns:
    print("\nYear of Detection Counts:")
    print(training_data['year'].value_counts())
else:
    print("\nNo 'year' column found in training data.")

# ----------------------------
# 3. Inspecting Test AOIs
# ----------------------------

# a. List All Test Shapefiles
test_shapefiles_dir = os.path.join('shp_test_AOIs', 'shp')

if not os.path.isdir(test_shapefiles_dir):
    print(f"Test shapefiles directory not found at: {test_shapefiles_dir}")
    sys.exit(1)

test_shapefiles = [f for f in os.listdir(
    test_shapefiles_dir) if f.endswith('.shp')]

print(f"\nFound {len(test_shapefiles)} Test Shapefiles:")
for shp in test_shapefiles:
    print(f"- {shp}")

# b. Load and Inspect a Specific Test Shapefile
if not test_shapefiles:
    print("No test shapefiles found to inspect.")
    sys.exit(1)

test_shapefile_name = test_shapefiles[0]
test_shapefile_path = os.path.join(test_shapefiles_dir, test_shapefile_name)

test_data = gpd.read_file(test_shapefile_path)
print(f"\nTest AOI Data Loaded Successfully: {test_shapefile_name}")

# c. Inspect the Test AOI Data
print("\nFirst Five Rows of Test AOI Data:")
print(test_data.head())

print("\nTest AOI Data Information:")
print(test_data.info())

print("\nTest AOI Data CRS:")
print(test_data.crs)


# e. Overlay Training Runways on Test AOI
if training_data.crs != test_data.crs:
    training_data_converted = training_data.to_crs(test_data.crs)
    print("\nConverted training data CRS to match test AOI CRS.")
else:
    training_data_converted = training_data

fig, ax = plt.subplots(figsize=(10, 10))
test_data.plot(ax=ax, color='lightgrey', edgecolor='black',
               alpha=0.5, label='Test AOI')
training_data_converted.plot(
    ax=ax, color='red', edgecolor='black', alpha=0.5, label='Training Runways')
plt.title('Training Runways Overlay on Test AOI')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

# f. Spatial Relationship Analysis
intersections = gpd.sjoin(training_data_converted,
                          test_data, how='inner', predicate='intersects')
print(
    f"\nNumber of Training Runways Intersecting {test_shapefile_name}: {len(intersections)}")

# ----------------------------
# 5. Summary and Relationships
# ----------------------------

print("\n=== Summary ===")
print("""
- **Training Data**: Contains polygons of known clandestine runways with attributes such as year of detection. Used to train the detection model.
- **Test AOIs**: Define areas where the model needs to detect potential runways for specified years.
- **Submission CSVs**:
    - `input.csv`: Labeled tile data for training/validation.
    - `SampleSubmission.csv`: Template showing the required submission format with a large number of rows.
    
**Relationships**:
- Train the model using training shapefiles and input.csv.
- Apply the model to test AOIs to predict runways.
- Generate a reduced submission file (~200,000 rows) based on predictions, focusing on detected runways and their buffer regions.
""")

Training shapefile not found at: pac_2024_training/pac_2024_training.shp


SystemExit: 1

/home/emil/Desktop/secret-runway-detection/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Path to your AOI shapefile
shapefile_path = 'shp_test_AOIs/shp/aoi_2021_04.shp'

# Load the shapefile
aoi = gpd.read_file(shapefile_path)

# Display basic information
print("=== Shapefile Information ===")
print(aoi.info())

# Display the first few rows
print("\n=== First Five Rows ===")
print(aoi.head())

# Print Coordinate Reference System (CRS)
print("\n=== Coordinate Reference System (CRS) ===")
print(aoi.crs)

# Plot the shapefile
aoi.plot(color='lightblue', edgecolor='black')
plt.title('AOI: aoi_2021_04.shp')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, Point  # Imported Point for centroid
import matplotlib.pyplot as plt
import contextily as ctx
import re
import os


def parse_tile_identifier(tile_id):
    """
    Parses the tile identifier to extract AOI year, month, row, and column.

    Parameters:
    - tile_id (str): Tile identifier in the format 'Tileaoi_YY_MM_row_col'

    Returns:
    - year (str): Four-digit year
    - month (str): Two-digit month
    - row (int): Tile row index
    - col (int): Tile column index
    """
    pattern = r'Tileaoi_(\d{2})_(\d{2})_(\d+)_(\d+)'
    match = re.match(pattern, tile_id)
    if not match:
        raise ValueError(
            "Tile identifier does not match the expected format 'Tileaoi_YY_MM_row_col'.")
    yy, mm, row, col = match.groups()
    year = f"20{yy}"
    month = mm
    return year, month, int(row), int(col)


def calculate_tile_extent(aoi_bounds, row, col, total_rows, total_cols):
    """
    Calculates the spatial extent (bounding box) of a specific tile within the AOI.

    Parameters:
    - aoi_bounds (tuple): (minx, miny, maxx, maxy) of the AOI
    - row (int): Tile row index
    - col (int): Tile column index
    - total_rows (int): Total number of tile rows in the AOI
    - total_cols (int): Total number of tile columns in the AOI

    Returns:
    - tile_polygon (Polygon): Shapely Polygon of the tile's bounding box
    """
    minx, miny, maxx, maxy = aoi_bounds
    tile_width = (maxx - minx) / total_cols
    tile_height = (maxy - miny) / total_rows

    tile_minx = minx + (col - 1) * tile_width
    tile_maxx = minx + col * tile_width
    tile_miny = miny + (row - 1) * tile_height
    tile_maxy = miny + row * tile_height

    tile_polygon = Polygon([
        (tile_minx, tile_miny),
        (tile_minx, tile_maxy),
        (tile_maxx, tile_maxy),
        (tile_maxx, tile_miny),
        # (tile_minx, tile_miny)
    ])

    return tile_polygon


def inspect_tile(tile_id, shapefiles_dir, total_rows, total_cols):
    """
    Inspects a specific tile by plotting it within the corresponding AOI shapefile.

    Parameters:
    - tile_id (str): Tile identifier (e.g., 'Tileaoi_21_03_1047_1231')
    - shapefiles_dir (str): Directory containing AOI shapefiles
    - total_rows (int): Total number of tile rows in the AOI
    - total_cols (int): Total number of tile columns in the AOI
    """
    # Parse the tile identifier
    year, month, row, col = parse_tile_identifier(tile_id)
    print(
        f"AOI Year: {year}, AOI Month: {month}, Tile Row: {row}, Tile Column: {col}")

    # Determine the AOI shapefile path
    aoi_filename = f"aoi_{year}_{month}.shp"
    aoi_path = os.path.join(shapefiles_dir, aoi_filename)

    if not os.path.exists(aoi_path):
        raise FileNotFoundError(f"AOI shapefile not found: {aoi_path}")

    # Load the AOI shapefile
    aoi_gdf = gpd.read_file(aoi_path)
    aoi_bounds = aoi_gdf.total_bounds  # [minx, miny, maxx, maxy]

    # Calculate the tile's spatial extent
    tile_polygon = calculate_tile_extent(
        aoi_bounds, row, col, total_rows, total_cols)

    # Create a GeoDataFrame for the tile
    tile_gdf = gpd.GeoDataFrame([{'geometry': tile_polygon}], crs=aoi_gdf.crs)

    # Reproject to Web Mercator for basemap compatibility
    aoi_gdf = aoi_gdf.to_crs(epsg=3857)
    tile_gdf = tile_gdf.to_crs(epsg=3857)

    # Check if the tile is within the AOI
    is_within = tile_gdf.within(aoi_gdf.unary_union).iloc[0]
    print(f"Is the tile within the AOI? {'Yes' if is_within else 'No'}")

    # ----------------------------
    # Added: Create and Plot Circle around Tile
    # ----------------------------
    # Calculate AOI dimensions
    aoi_width = aoi_bounds[2] - aoi_bounds[0]
    aoi_height = aoi_bounds[3] - aoi_bounds[1]
    aoi_average = (aoi_width + aoi_height) / 2

    # Define circle radius as 1/8 of the AOI's average dimension
    circle_radius = aoi_average / 8

    # Get the centroid of the tile
    tile_centroid = tile_gdf.geometry.centroid.iloc[0]

    # Create a circular polygon around the centroid
    circle = tile_centroid.buffer(circle_radius)

    # Create a GeoDataFrame for the circle
    circle_gdf = gpd.GeoDataFrame([{'geometry': circle}], crs=aoi_gdf.crs)

    # ----------------------------
    # Added: Create and Plot Circle around Tile
    # ----------------------------
    # Calculate AOI dimensions
    aoi_width = aoi_bounds[2] - aoi_bounds[0]
    aoi_height = aoi_bounds[3] - aoi_bounds[1]
    aoi_average = (aoi_width + aoi_height) / 2

    # Define circle radius as 1/8 of the AOI's average dimension
    circle_radius = aoi_average / 8

    # Get the centroid of the tile
    tile_centroid = tile_gdf.geometry.centroid.iloc[0]

    # Create a circular polygon around the centroid
    circle = tile_centroid.buffer(circle_radius)

    # Create a GeoDataFrame for the circle
    circle_gdf = gpd.GeoDataFrame([{'geometry': circle}], crs=aoi_gdf.crs)

    # ----------------------------
    # Plot AOI, Tile, Circle, and Zoomed Tile
    # ----------------------------
    fig, axes = plt.subplots(1, 2, figsize=(24, 12))

    # Plot 1: Full AOI with Tile and Circle
    aoi_gdf.plot(ax=axes[0], color='lightblue',
                 edgecolor='black', label='AOI', alpha=0.5)
    if is_within:
        tile_gdf.plot(ax=axes[0], color='red',
                      edgecolor='black', alpha=0.5, label='Tile')
    else:
        tile_gdf.plot(ax=axes[0], color='gray', edgecolor='black',
                      alpha=0.5, label='Tile (Outside AOI)')
    circle_gdf.plot(ax=axes[0], facecolor='none',
                    edgecolor='yellow', linewidth=2, label='Tile Circle')
    ctx.add_basemap(axes[0], source=ctx.providers.Esri.WorldImagery, zoom=12)
    axes[0].set_title(f'Inspecting {tile_id} within {aoi_filename}')
    axes[0].set_xlabel('Easting')
    axes[0].set_ylabel('Northing')
    axes[0].legend()

    # Plot 2: Zoomed-in Tile
    tile_gdf.plot(ax=axes[1], color='lightblue',
                  edgecolor='black', alpha=0.5, label='Tile')
    # bounds = tile_polygon.bounds
    # padding_coef = 1
    # axes[1].set_xlim(bounds[0] - padding_coef * (bounds[2] - bounds[0]), bounds[2] + padding_coef * (bounds[2] - bounds[0]))
    # axes[1].set_ylim(bounds[1] - padding_coef * (bounds[3] - bounds[1]), bounds[3] + padding_coef * (bounds[3] - bounds[1]))
    ctx.add_basemap(
        axes[1],
        source=ctx.providers.Esri.WorldImagery,
        zoom=12)  # Higher zoom level for detail
    axes[1].set_title(f'Zoomed-in View of {tile_id}')
    axes[1].set_xlabel('Easting')
    axes[1].set_ylabel('Northing')

    plt.tight_layout()
    plt.show()

    return is_within

# ----------------------------
# Example Usage
# ----------------------------


if __name__ == "__main__":
    # Define the tile identifier you want to inspect
    tile_identifier = 'Tileaoi_21_03_1047_1231'  # Replace with your tile ID

    # Define the directory containing AOI shapefiles
    shapefiles_dir = 'shp_test_AOIs/shp'  # Adjust if different

    # Define the total number of rows and columns in the AOI grid
    # **Important:** Replace these with actual numbers based on your tiling system
    total_tile_rows = 2000  # Example value
    total_tile_cols = 2000  # Example value

    # Inspect the tile
    try:
        within_aoi = inspect_tile(
            tile_identifier, shapefiles_dir, total_tile_rows, total_tile_cols)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
import geopandas as gpd

# Path to one of your AOI shapefiles
# Replace with the desired shapefile
aoi_shapefile = 'shp_test_AOIs/shp/aoi_2021_03.shp'

# Load the shapefile
aoi_gdf = gpd.read_file(aoi_shapefile)

# Inspect the first few rows and columns
print(aoi_gdf.head())
print(aoi_gdf.columns)
print(aoi_gdf)

In [ ]:
import geopandas as gpd
import os
import math
import pandas as pd

def lon_to_utm_zone(lon):
    """Calculate the UTM zone number from longitude, capped between 1 and 60."""
    zone = int((lon + 180) / 6) + 1
    zone = min(max(zone, 1), 60)
    return zone

def get_utm_crs(lat, lon):
    """Return the EPSG code for the UTM CRS based on latitude and longitude."""
    zone = lon_to_utm_zone(lon)
    if lat >= 0:
        epsg_code = 32600 + zone  # Northern hemisphere
    else:
        epsg_code = 32700 + zone  # Southern hemisphere
    return f'EPSG:{epsg_code}'

def calculate_aoi_metrics(shp_dir):
    """
    Calculate area, width, and height for each shapefile in the directory.

    Parameters:
    - shp_dir (str): Path to the directory containing shapefiles.

    Returns:
    - pd.DataFrame: DataFrame containing metrics for each AOI.
    """
    # List all .shp files in the directory
    shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

    # Initialize a list to store results
    results = []

    # Process each shapefile
    for shp_file in shp_files:
        # Full path to the shapefile
        shp_path = os.path.join(shp_dir, shp_file)
        
        try:
            # Read the shapefile using GeoPandas
            gdf = gpd.read_file(shp_path)
        except Exception as e:
            print(f"Error reading {shp_file}: {e}")
            continue

        # Ensure the GeoDataFrame has a CRS
        if gdf.crs is None:
            print(f"Shapefile {shp_file} has no CRS. Skipping.")
            continue

        # Reproject to WGS84 if not already
        if gdf.crs.to_string() != 'EPSG:4326':
            gdf = gdf.to_crs('EPSG:4326')

        # Select the first geometry (assuming one AOI per shapefile)
        try:
            runway = gdf.iloc[0]
            geometry = runway.geometry
        except Exception as e:
            print(f"Error processing geometry in {shp_file}: {e}")
            continue

        # Calculate centroid
        centroid = geometry.centroid
        lon, lat = centroid.x, centroid.y

        # Get appropriate UTM CRS
        utm_crs = get_utm_crs(lat, lon)

        try:
            # Reproject to UTM CRS for accurate measurements
            gdf_projected = gdf.to_crs(utm_crs)
        except Exception as e:
            print(f"Error reprojecting {shp_file} to {utm_crs}: {e}")
            continue

        # Calculate area in square meters
        try:
            area = gdf_projected['geometry'].area.iloc[0]
        except Exception as e:
            print(f"Error calculating area for {shp_file}: {e}")
            area = None

        # Get the bounding box dimensions (minx, miny, maxx, maxy)
        try:
            minx, miny, maxx, maxy = gdf_projected.total_bounds
            width = maxx - minx  # Width in meters
            height = maxy - miny  # Height in meters
        except Exception as e:
            print(f"Error calculating dimensions for {shp_file}: {e}")
            width = height = None

        # Append the results
        results.append({
            'Shapefile': shp_file,
            'Area (sq m)': round(area, 2) if area else None,
            'Width (m)': round(width, 2) if width else None,
            'Height (m)': round(height, 2) if height else None
        })

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    return df

if __name__ == "__main__":
    # Directory containing the shapefiles
    shp_dir = 'shp_test_AOIs/shp'  # Update this path if necessary

    # Calculate metrics
    metrics_df = calculate_aoi_metrics(shp_dir)

    # Display the results
    print(metrics_df)

    # Optionally, save to a CSV file
    metrics_df.to_csv('aoi_metrics.csv', index=False)
    print("Metrics have been saved to 'aoi_metrics.csv'.")


In [ ]:
# inspect_crs.py

import os
import geopandas as gpd
from shapely.geometry import Polygon
import pyproj
import warnings

def find_geospatial_files(directory, extensions=['.shp', '.geojson']):
    """
    Recursively find all shapefiles and GeoJSON files in a directory.
    Returns a list of file paths.
    """
    geospatial_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                geospatial_files.append(os.path.join(root, file))
    return geospatial_files

def get_shapefile_prj(shp_path):
    """
    Given a shapefile path, return the path to its corresponding .prj file.
    """
    base, _ = os.path.splitext(shp_path)
    prj_path = base + '.prj'
    if os.path.exists(prj_path):
        return prj_path
    else:
        return None

def read_prj_crs(prj_path):
    """
    Read the CRS from a .prj file using pyproj.
    Returns the EPSG code if possible, else returns the full WKT string.
    """
    with open(prj_path, 'r') as prj_file:
        prj_text = prj_file.read()
    try:
        crs = pyproj.CRS.from_wkt(prj_text)
        if crs.to_epsg():
            return f"EPSG:{crs.to_epsg()}"
        else:
            return crs.to_string()
    except Exception as e:
        warnings.warn(f"Unable to parse CRS from {prj_path}: {e}")
        return "Unknown CRS"

def inspect_crs(files, target_crs='EPSG:32718'):
    """
    Inspect and print CRS of given geospatial files.
    For shapefiles, ensure that the .prj file matches the CRS inferred by GeoPandas.
    For GeoJSON files, confirm they are in the expected CRS (default EPSG:4326).
    Warns if discrepancies are found.
    """
    if not files:
        print("No geospatial files found to inspect.")
        return
    
    summary = {
        'Shapefiles': {'Total': 0, 'With PRJ': 0, 'Without PRJ': 0, 'CRS Match': 0, 'CRS Mismatch': 0},
        'GeoJSON': {'Total': 0, 'Expected CRS': 0, 'Different CRS': 0}
    }
    
    print("\n=== CRS Inspection Report ===\n")
    
    for file in files:
        file_lower = file.lower()
        if file_lower.endswith('.shp'):
            summary['Shapefiles']['Total'] += 1
            prj_path = get_shapefile_prj(file)
            print(f"Inspecting Shapefile: {file}")
            
            if prj_path:
                summary['Shapefiles']['With PRJ'] += 1
                prj_crs = read_prj_crs(prj_path)
                print(f"  .prj CRS: {prj_crs}")
            else:
                summary['Shapefiles']['Without PRJ'] += 1
                prj_crs = "Not Defined"
                print("  Warning: Missing .prj file.")
            
            try:
                gdf = gpd.read_file(file)
                gdf_crs = gdf.crs.to_string() if gdf.crs else "Not Defined"
                print(f"  GeoPandas Inferred CRS: {gdf_crs}")
                
                if prj_crs != "Not Defined":
                    if gdf_crs == prj_crs:
                        summary['Shapefiles']['CRS Match'] += 1
                        print("  Status: ✅ CRS Match\n")
                    else:
                        summary['Shapefiles']['CRS Mismatch'] += 1
                        print("  Status: ❌ CRS Mismatch\n")
                        warnings.warn(f"CRS mismatch in {file}: .prj indicates {prj_crs}, but GeoPandas infers {gdf_crs}.")
                else:
                    print("  Status: ❌ CRS Undefined\n")
                    warnings.warn(f"CRS undefined for {file}.")
            
            except Exception as e:
                print(f"  Error reading {file}: {e}\n")
                warnings.warn(f"Error reading {file}: {e}")
        
        elif file_lower.endswith('.geojson'):
            summary['GeoJSON']['Total'] += 1
            print(f"Inspecting GeoJSON: {file}")
            try:
                gdf = gpd.read_file(file)
                gdf_crs = gdf.crs.to_string() if gdf.crs else "Not Defined"
                print(f"  Inferred CRS: {gdf_crs}")
                
                if 'EPSG:4326' == gdf_crs:
                    summary['GeoJSON']['Expected CRS'] += 1
                    print("  Status: ✅ Expected CRS (EPSG:4326)\n")
                else:
                    summary['GeoJSON']['Different CRS'] += 1
                    print("  Status: ❌ Different CRS than Expected (EPSG:4326)\n")
                    warnings.warn(f"GeoJSON {file} has CRS {gdf_crs}, expected EPSG:4326.")
            
            except Exception as e:
                print(f"  Error reading {file}: {e}\n")
                warnings.warn(f"Error reading {file}: {e}")
    
    # Summary Report
    print("\n=== Summary ===\n")
    print("Shapefiles:")
    print(f"  Total Shapefiles: {summary['Shapefiles']['Total']}")
    print(f"  With .prj: {summary['Shapefiles']['With PRJ']}")
    print(f"  Without .prj: {summary['Shapefiles']['Without PRJ']}")
    print(f"  CRS Match: {summary['Shapefiles']['CRS Match']}")
    print(f"  CRS Mismatch: {summary['Shapefiles']['CRS Mismatch']}\n")
    
    print("GeoJSON Files:")
    print(f"  Total GeoJSON Files: {summary['GeoJSON']['Total']}")
    print(f"  Expected CRS (EPSG:4326): {summary['GeoJSON']['Expected CRS']}")
    print(f"  Different CRS: {summary['GeoJSON']['Different CRS']}\n")
    
    # Warnings
    if summary['Shapefiles']['CRS Mismatch'] > 0 or summary['GeoJSON']['Different CRS'] > 0:
        print("Warnings were issued for CRS discrepancies. Please review the warnings above.")
    else:
        print("All inspected files have consistent and expected CRSs.")

def main():
    # Directories to inspect
    training_dirs = ['pac_2024_training/', 'shp_test_AOIs/']
    
    # Find all geospatial files in these directories
    all_files = []
    for directory in training_dirs:
        if os.path.exists(directory):
            print(f"Searching in Directory: {directory}")
            files = find_geospatial_files(directory)
            if files:
                print(f"Found {len(files)} geospatial file(s) in {directory}.\n")
                all_files.extend(files)
            else:
                print(f"No geospatial files found in {directory}.\n")
        else:
            print(f"Directory {directory} does not exist.\n")
    
    # Inspect CRS of all found files
    inspect_crs(all_files)

if __name__ == "__main__":
    main()
